# 名字、名字空间和作用域
- <font color= red>变量</font>的定义本质上就是建立<font color= red>名字</font>和<font color= red>对象</font>之间的约束关系，名字是一个符号。可以通过变量赋值、创建函数、类等定义变量，完成名字和对象之间的绑定。

- <font color= red>作用域</font>可以理解为一段程序的正文区域，在这个区域里面定义的变量是有意义的，然而一旦出了这个区域，就无效了。因此作用域是由源程序的文本所决定的，是静态结构。

- <font color= red>名字空间</font>本质上是一个字典，键是名字、值是对象。


Python具有<font color= red>静态作用域</font>(词法作用域)，而<font color= red>名字空间则是作用域的动态体现</font>，一个由程序文本定义的作用域在Python运行时会转化为一个名字空间、即一个PyDictObject对象。而进入一个函数，显然进入了一个新的作用域，因此函数在执行时，会创建一个名字空间。


- Python定义了一个最顶层的作用域，也就是<font color= red>builtin作用域</font>，像内置函数、内建对象都在<font color= plum>builtin名字空间</font>里面；
- 模块对应的源文件本身也有相应的作用域，即<font color= red>global作用域</font>。所以解释器在运行这个文件的时候，也会为其创建一个名字空间，而这个名字空间就是<font color= plum>global名字空间</font>。它里面的变量是全局的，或者说是模块级别的，在当前文件的任意位置都可以直接访问。
- 函数、类也有作用域，这个作用域称为<font color= red>local作用域</font>，对应<font color= plum>local名字空间</font>。


- 作用域是层层嵌套的；
- 内层作用域可以访问外层作用域；
- 外层作用域无法访问内层作用域；
- 查找元素会依次从当前作用域向外查找。

In [1]:
a = 3

def foo():
    a = 4
    print(a)
    
def hoo():
    print(a)

foo()      # 4  
hoo()      # 3
print(a)   # 3    

4
3
3


In [2]:
class A:
    a = 1


class B:
    a = 2
    print(A.a)  # 1  属性访问,不同作用域，则需要通过访问修饰符 . 进行属性访问
    print(a)    # 2  直接访问,位于同一个作用域的代码可以直接访问作用域中出现的名字

1
2



<font color= red>获取名字空间</font>，Python也提供了相应的内置函数：

locals函数：获取当前作用域的local名字空间，local名字空间也称为局部名字空间；

globals函数：获取当前作用域的global名字空间，global名字空间也称为全局名字空间；

__builtins__函数：或者import builtins，获取当前作用域的builtin名字空间，builtint名字空间也称为内置名字空间；

### global名字空间
<font color= red>global名字空间是唯一的</font>，在任何地方调用globals()得到的都是global名字空间，正如你在任何地方都可以访问到全局变量一样。
我们之前提到，名字空间是一个字典，变量和对象会以键值对的形式存在里面。因此在此处，<font color= red>通过往global名字空间里面插入一个键值对完全等价于定义一个全局变量</font>。

In [3]:
# 直接向global名字空间添加键值对，等价于定义一个全局变量、并和对象绑定起来
globals()["name"] = "古明地觉"
print(name)  # 古明地觉

#在函数中向global名字空间添加键值对
def f1():
    def f2():
        def f3():
            globals()["age"] = 16
        return f3
    return f2
f1()()()
print(age)  # 16

古明地觉
16


### local名字空间
对于local名字空间来说，它也对应一个字典，显然这个字典就不是全局唯一的了，每一个局部作用域都会对应自身的local名字空间。

In [4]:
def f():
    name = "夏色祭"
    age = 16
    return locals()


def g():
    name = "神乐mea"
    age = 38
    return locals()


print(locals() == globals())  # True
print(f())  # {'name': '夏色祭', 'age': 16}
print(g())  # {'name': '神乐mea', 'age': 38}

True
{'name': '夏色祭', 'age': 16}
{'name': '神乐mea', 'age': 38}


In [5]:
def f1():
    locals()["name_1 "] = "夏色祭"
    try:
        print(name_1)
    except Exception as e:
        print(e)

f1()  # name 'name_1' is not defined

name 'name_1' is not defined


- 对于函数来讲，内部的变量是通过静态方式存储和访问的，因为局部作用域中存在哪些变量在编译的时候就已经确定了。因此<font color= red>不可以通过向local名字空间添加键值对的方式定义局部变量</font>。
- 对于全局变量来讲，因为全局变量会一直在变，需要使用字典来动态维护，因此变量的创建是通过向字典添加键值对的方式实现的。

### builtin名字空间
对于builtin名字空间，它也是一个字典。当local空间、global空间都没有的时候，会去builtin空间查找。问题来了，builtin名字空间如何获取呢？答案是使用builtins模块，通过builtins. __dict__即可拿到builtin名字空间。

In [ ]:
# 等价于__builtins__
import builtins

# local, global名字空间没有定义dict
# 将builtin空间的dict改成123,那么此时获取的dict就是123,因为是从内置作用域中获取的
print(builtins.dict) # <class 'dict'>
builtins.dict = 123
print(dict)  # 123

#先在global名字空间创建全局变量，此时改变全局空间，但不影响builtin空间
str = 123
print(str)  # 123
print(builtins.str)  # <class 'str'>

### LEB规则 与 LEGB规则

LGB：local - global - builtin （局部 - 全局 - 内置名字空间）

<font color= red>LEGB：local - enclosing - global - builtin （局部 - 闭包 - 全局 - 内置名字空间）</font>

从Python2.2开始，引入了嵌套函数，因此内层函数找不到某个变量时应先去外层函数找，而不是直接跑到global空间里面找。此处E表示enclosing，代表直接外围作用域。

在执行f = foo()的时候，会执行函数foo中的def bar():语句，这个时候解释器会将a=2与函数bar捆绑在一起，然后返回，这个捆绑起来的整体就叫做闭包。所以：闭包 = 内层函数 + 引用的外层作用域


In [6]:
a = 1

def foo():
    a = 2
    def bar():
        print(a)
    return bar

f = foo()
f()      # 2

2


虽然每个模块(源文件)内部的作用域规则有点复杂，因为要遵循LEGB；但模块与模块之间的作用域还是划分的很清晰的，就是相互独立。<font color= red>名称引用虽然是LEGB规则，但是无论如何都无法越过自身所在的模块。</font>

如下例所示，即使我们把a导入了进来，但是a.py里面的内容依旧是处于一个模块里面。print(name)在a.py里面，而变量name被定义在b.py里面，所以不可能跨过模块a的作用域去访问模块b里面的name，因此在执行 import a 的时候会抛出 NameError。

In [7]:
# a.py
print(name)

# b.py
name = "夏色祭"
import a # NameError

古明地觉


ModuleNotFoundError: No module named 'a'

### global 关键字
在函数内对某个名字进行赋值操作，这个名字就会在作用域内可见，出现在local名字空间中，同时会遮蔽外层作用域中相同的名字。Python也为我们精心准备了<font color= red>global关键字，可以在函数内部修改全局变量</font>。比如函数内部出现了global a，就表示我后面的a是全局的，直接到global名字空间里面去找，不要在local空间里面找了。
### nonlocal 关键字
那么如果想要<font color= red>在内层函数中修改外层函数中的变量a，不是全局的a，此时可以使用nonlocal关键字</font>，注意，使用nonlocal的时候必须是在内层函数里面。

In [8]:
# global关键字
a = 1

def bar():
    def foo():
        global a
        a = 2
    return foo

bar()()
print(a)  # 2

2


In [9]:
# nonlocal关键字
a = 1

def bar():
    a = 2
    def foo():
        nonlocal a
        a = "xxx"
    foo()
    print(a)
    return foo

bar()()   #xxx
print(a)  # 1

xxx
1


### eval()和exec()函数中的局部、全局名字空间参数
eval(expression[, globals[, locals]]) # 只能执行计算数学表达式的功能

exec(expression[, globals[, locals]]) # 能够动态地执行复杂的python代码

In [10]:
# 如果不指定，默认当前所在的名字空间,显然此时是全局名字空间
exec("name = '古明地觉'")
print(name)  # 古明地觉

# 将 dct 作为全局名字空间,这里我们没有指定第三个参数，也就是局部名字空间.
dct = {}
exec("name = 'satori'", dct)
print(dct["name"])  # satori

古明地觉
satori


### 小结
在 Python 中，名字是指代变量的一个符号，它的具体含义由的可见范围由作用域决定，而作用域由语法静态划分，划分规则提炼如下：

- .py文件(模块)最外层为全局作用域；
- 遇到函数定义，函数体形成子作用域；
- 遇到类定义，类定义体形成子作用域；
- 名字仅在其作用域以内可见；
- 全局作用域对其他所有作用域可见；
- 函数作用域对其直接子作用域可见，并且可以传递(闭包)。


与作用域相对应， Python在运行时借助PyDictObject对象保存作用域中的名字，构成动态的名字空间 。这样的名字空间总共有 4 个：

- 局部名字空间(local)：不同的函数，局部名字空间不同，可以通过调用 locals 获取；
- 闭包名字空间(enclosing)；
- 全局名字空间(global)：全局唯一，可以通过调用 globals 获取；
- 内置名字空间(builtin)：可以通过调用 __builtins__.__dict__ 获取；


查找名字时会按照LEGB规则查找，但是注意：无法跨越文件本身，也就是按照自身文件的LEGB。如果属性查找都找到builtin空间了，那么证明这已经是最后的倔强。如果builtin空间再找不到，那么就只能报错了，不可能跑到其它文件中找。